In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time

# Setup WebDriver with incognito mode and user-agent
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36")
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the Amazon home page
url = "https://www.amazon.de/"
driver.get(url)
time.sleep(3)  # Allow initial load
 
try:
    # Attempt to open location settings if not already open
    location_link = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "nav-global-location-popover-link"))
    )
    location_link.click()
except TimeoutException:
    print("Location settings already open or link not found; proceeding.")
 
try:
    # Click the 'Change' link to change the postal code
    change_link = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "GLUXChangePostalCodeLink"))
    )
    change_link.click()
except TimeoutException:
    print("Change link not found; maybe already in change mode.")
 
try:
    # Wait for input to be visible and enter the new postal code
    zip_input = WebDriverWait(driver, 5).until(
        EC.visibility_of_element_located((By.ID, "GLUXZipUpdateInput"))
    )
    zip_input.clear()
    zip_input.send_keys("01067")
    # Using ENTER key as an alternative to clicking submit
    zip_input.send_keys(Keys.ENTER)
except (TimeoutException, NoSuchElementException):
    print("Postal code input or submit action failed.")
 
try:
    # Finalize the location change by clicking 'Done'
    done_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@name='glowDoneButton' and contains(@id, 'announce')]"))
    )
    done_button.click()
except TimeoutException:
    print("Done button not found or not clickable; maybe location update not needed.")
 
# List of keywords to search
keywords = ["digestive enzyme",  "Digestive Enzymes", "enzymedica", "digest basic", "digest gold", "gluten ease", "glutenease",
            "digest basic enzymedica", "gluten digest", "verdauungsenzym", "glutenfreie lebensmittel", "fettverbrenner", 
            "enzyme tabletten", "galactosidase"]
 
# Perform searches for each keyword and collect data
all_products = []
for keyword in keywords:
    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "twotabsearchtextbox"))
        )
        search_box.clear()
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.RETURN)
        time.sleep(3)  # Wait for search results to load
 
        # Scroll through the search results to load all products
        while True:
            last_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Wait to load page
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
 
        # Collect all product links, types, and keywords from the current keyword search
        product_entries = driver.find_elements(By.CSS_SELECTOR, ".s-result-item")
        for entry in product_entries:
            try:
                link_element = entry.find_element(By.CSS_SELECTOR, "a.a-link-normal.a-text-normal")
                link = link_element.get_attribute('href')
                is_paid = "paid" if "Ad" in entry.get_attribute("class") else "organic"
                all_products.append({"keyword": keyword, "url": link, "type": is_paid})
            except NoSuchElementException:
                continue  # If element not found, skip to the next one
    except Exception as e:
        print(f"Error during search for '{keyword}':", e)
 
# Visit each product page from collected links and gather data
results = []
for product in all_products:
    try:
        driver.get(product["url"])
        time.sleep(3)  # Allow page to load
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scroll to the bottom of the page
        time.sleep(1)  # Allow any lazy-loaded elements to load
        asin_row = driver.find_element(By.XPATH, "//th[contains(text(), 'ASIN')]/following-sibling::td")
        title = driver.find_element(By.ID, 'productTitle').text.strip()
        product.update({"ASIN": asin_row.text, "title": title})
        results.append(product)
    except Exception as e:
        print(f"Could not extract data from {product['url']}: {str(e)}")
 
# Convert results to a DataFrame
df = pd.DataFrame(results)
# Group by 'type' and 'keyword', then rank each product
df['rank'] = df.groupby(['type', 'keyword']).cumcount() + 1
 
# Print and save the DataFrame
print(df)
df.to_csv("germany_amazon_product_details.csv", index=False)
 
# Close the driver
driver.quit()

from datetime import datetime
import os
 
filename = "/Users/z.an/Desktop/Amazon_Germany_Positioning.xlsx"
 
# ASIN list
asin_info = pd.DataFrame({
    'ASIN': [
        'B0C6M5Q21P', 'B0C6M4VGHY', 'B0C6M4H3XZ', 'B0C6M497SS', 'B0C6L1MHRJ',
        'B0C6L38JTS', 'B0C6L2QZSB', 'B0C6L2473Z', 'B0C6L2M81X', 'B0C62DWSDL',
        'B0C6M5H6LF', 'B0C6L17CJT', 'B0C6L4T39J', 'B0C6L31VKN'
    ],
    'SHORT NAME': [
        'DBB-90-EU', 'DB-90-EU', 'DB-180-EU', 'DGB-90-EU', 'DG-90-EU',
        'DG-180-EU', 'D-90-EU', 'D-180-EU', 'PB-90-EU', 'LG-60-EU',
        'LG-120-EU', 'L-90-EU', 'DS-90-EU', 'GE-60-EU'
    ],
    'Market': [
        'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE', 'GE'
    ]
})
 
# Assuming 'df' contains the scraped data, merge it with the ASIN List to include additional information
df_with_asin_info = pd.merge(df, asin_info, on='ASIN', how='right')
 
# Pivot the DataFrame
pivoted_df = df_with_asin_info.pivot_table(index=['keyword', 'ASIN', 'SHORT NAME', 'Market'], columns='type', values='rank', fill_value=0).reset_index()
 
# Add a 'Date' column with the current date
current_date = datetime.now().strftime('%Y-%m-%d')
pivoted_df['Date'] = current_date
 
# Convert 'organic' and 'paid' rankings to integers
pivoted_df['organic'] = pivoted_df['organic'].astype(int)
pivoted_df['paid'] = pivoted_df['paid'].astype(int)
 
# Check if the file exists and append or create new file
if os.path.exists(filename):
    # Load the existing data
    old_data = pd.read_excel(filename)
    # Combine old data with new data, avoiding duplicates based on 'keyword', 'ASIN', 'SHORT NAME', 'Market', and 'Date'
    combined_data = pd.concat([old_data, pivoted_df]).drop_duplicates(subset=['keyword', 'ASIN', 'SHORT NAME', 'Market', 'Date'])
else:
    # Use new data if file doesn't exist
    combined_data = pivoted_df
 
# Save the combined data back to Excel, overwriting the old file
combined_data.to_excel(filename, index=False)
 
# Display the updated DataFrame
combined_data


Change link not found; maybe already in change mode.
Done button not found or not clickable; maybe location update not needed.
Error during search for 'digestive enzyme': Message: element not interactable
  (Session info: chrome=131.0.6778.85)
Stacktrace:
	GetHandleVerifier [0x00007FF71A256CB5+28821]
	(No symbol) [0x00007FF71A1C3840]
	(No symbol) [0x00007FF71A0655B9]
	(No symbol) [0x00007FF71A0B2D14]
	(No symbol) [0x00007FF71A0B0DAD]
	(No symbol) [0x00007FF71A0DF2FA]
	(No symbol) [0x00007FF71A0AC3F6]
	(No symbol) [0x00007FF71A0DF510]
	(No symbol) [0x00007FF71A0FF412]
	(No symbol) [0x00007FF71A0DF0A3]
	(No symbol) [0x00007FF71A0AA778]
	(No symbol) [0x00007FF71A0AB8E1]
	GetHandleVerifier [0x00007FF71A58FCAD+3408013]
	GetHandleVerifier [0x00007FF71A5A741F+3504127]
	GetHandleVerifier [0x00007FF71A59B5FD+3455453]
	GetHandleVerifier [0x00007FF71A31BDBB+835995]
	(No symbol) [0x00007FF71A1CEB5F]
	(No symbol) [0x00007FF71A1CA814]
	(No symbol) [0x00007FF71A1CA9AD]
	(No symbol) [0x00007FF71A1BA19

KeyError: 'type'